# 📝 Sumarizace a generování textu

**Autor:** Praut s.r.o. - AI Integration & Business Automation

## Co se naučíte:
- Automatická sumarizace dokumentů
- Generování textu s různými modely
- Tvorba reportů a shrnutí
- Automatizace tvorby obsahu

In [ ]:
!pip install -q transformers accelerate torch sentencepiece

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = 0 if torch.cuda.is_available() else -1
print(f"🖥️ Device: {'GPU' if device == 0 else 'CPU'}")

## 1. Extraktivní vs. Abstraktivní sumarizace

- **Extraktivní**: Vybírá klíčové věty z původního textu
- **Abstraktivní**: Generuje nový text shrnující obsah

In [ ]:
# BART model pro abstraktivní sumarizaci
summarizer = pipeline("summarization", 
                      model="facebook/bart-large-cnn",
                      device=device)

clanek = """
Artificial intelligence is rapidly transforming the business landscape across industries. 
Companies are increasingly adopting AI technologies to streamline operations, enhance 
customer experiences, and gain competitive advantages. Machine learning algorithms can 
analyze vast amounts of data to identify patterns and make predictions that would be 
impossible for humans to detect manually.

In the manufacturing sector, AI-powered robots are improving production efficiency and 
reducing errors. Predictive maintenance systems can anticipate equipment failures before 
they occur, saving companies millions in downtime costs. Quality control processes are 
being automated using computer vision, ensuring consistent product standards.

The financial industry is leveraging AI for fraud detection, risk assessment, and 
algorithmic trading. Banks are using chatbots to handle routine customer inquiries, 
freeing up human agents for more complex issues. Personalized financial advice is 
being delivered through AI-driven robo-advisors.

Healthcare is another sector seeing significant AI adoption. Diagnostic systems can 
analyze medical images with accuracy matching or exceeding human specialists. Drug 
discovery processes are being accelerated through AI simulations. Patient monitoring 
systems can detect anomalies and alert medical staff in real-time.
"""

souhrn = summarizer(clanek, max_length=100, min_length=30, do_sample=False)

print(f"📄 Původní text: {len(clanek)} znaků")
print(f"📋 Souhrn: {len(souhrn[0]['summary_text'])} znaků\n")
print(f"✨ {souhrn[0]['summary_text']}")

## 2. Různé délky sumarizace

In [ ]:
# Krátký souhrn (1-2 věty)
kratky = summarizer(clanek, max_length=50, min_length=20)
print("📌 KRÁTKÝ souhrn:")
print(f"   {kratky[0]['summary_text']}\n")

# Střední souhrn
stredni = summarizer(clanek, max_length=100, min_length=50)
print("📝 STŘEDNÍ souhrn:")
print(f"   {stredni[0]['summary_text']}\n")

# Dlouhý souhrn
dlouhy = summarizer(clanek, max_length=200, min_length=100)
print("📄 DLOUHÝ souhrn:")
print(f"   {dlouhy[0]['summary_text']}")

## 3. T5 model - flexibilnější sumarizace

In [ ]:
# T5 model
t5_summarizer = pipeline("summarization", 
                         model="t5-base",
                         device=device)

# T5 používá prefix "summarize:"
text_pro_t5 = clanek

t5_souhrn = t5_summarizer(text_pro_t5, max_length=80, min_length=20)
print("🔷 T5 souhrn:")
print(f"   {t5_souhrn[0]['summary_text']}")

## 4. Generování textu

In [ ]:
# GPT-2 pro generování textu
generator = pipeline("text-generation", 
                     model="gpt2-medium",
                     device=device)

prompt = "The future of artificial intelligence in business will"

# Generování s různými parametry
print("🎲 Kreativní generování (vysoká temperature):")
kreativni = generator(prompt, max_length=100, temperature=0.9, do_sample=True, num_return_sequences=1)
print(f"   {kreativni[0]['generated_text']}\n")

print("🎯 Konzervativní generování (nízká temperature):")
konzervativni = generator(prompt, max_length=100, temperature=0.3, do_sample=True, num_return_sequences=1)
print(f"   {konzervativni[0]['generated_text']}")

In [ ]:
# Více variant
print("📚 Vícenásobné generování (3 varianty):\n")

varianty = generator(
    "Automation in business helps companies to",
    max_length=60,
    num_return_sequences=3,
    do_sample=True,
    temperature=0.7
)

for i, var in enumerate(varianty, 1):
    print(f"Varianta {i}: {var['generated_text']}\n")

## 5. Praktická automatizace: Generátor newsletteru

In [ ]:
def generuj_newsletter_sekci(tema, body):
    """Generuje sekci newsletteru z bullet pointů."""
    
    # Shrnutí bodů
    text = f"{tema}. " + " ".join(body)
    souhrn = summarizer(text, max_length=100, min_length=30)
    
    return souhrn[0]['summary_text']

# Data pro newsletter
newsletter_data = {
    "AI novinky": [
        "OpenAI released GPT-4 with improved reasoning capabilities.",
        "Google announced Gemini multimodal AI model.",
        "Meta open-sourced Llama 3 for research and commercial use."
    ],
    "Produktové updaty": [
        "New dashboard with real-time analytics is now available.",
        "Integration with Slack and Teams has been improved.",
        "Mobile app now supports offline mode."
    ]
}

print("📰 AUTOMATICKY GENEROVANÝ NEWSLETTER\n")
print("=" * 50)

for sekce, body in newsletter_data.items():
    souhrn = generuj_newsletter_sekci(sekce, body)
    print(f"\n📌 {sekce.upper()}")
    print(f"   {souhrn}")

## 6. Sumarizace dlouhých dokumentů

In [ ]:
def sumarizuj_dlouhy_dokument(text, max_chunk_length=1000):
    """Sumarizuje dlouhý dokument po částech."""
    
    # Rozdělení na části (paragraphs)
    paragrafy = [p.strip() for p in text.split('\n\n') if p.strip()]
    
    # Sloučení do chunků
    chunky = []
    current_chunk = ""
    
    for para in paragrafy:
        if len(current_chunk) + len(para) < max_chunk_length:
            current_chunk += para + " "
        else:
            if current_chunk:
                chunky.append(current_chunk.strip())
            current_chunk = para + " "
    
    if current_chunk:
        chunky.append(current_chunk.strip())
    
    # Sumarizace každého chunku
    souhrny = []
    for i, chunk in enumerate(chunky):
        if len(chunk) > 100:  # Minimální délka pro sumarizaci
            souhrn = summarizer(chunk, max_length=60, min_length=20)
            souhrny.append(souhrn[0]['summary_text'])
            print(f"   ✓ Část {i+1}/{len(chunky)} zpracována")
    
    # Finální souhrn
    if len(souhrny) > 1:
        combined = " ".join(souhrny)
        final = summarizer(combined, max_length=150, min_length=50)
        return final[0]['summary_text']
    elif souhrny:
        return souhrny[0]
    return "Text je příliš krátký pro sumarizaci."

# Test na dlouhém dokumentu
dlouhy_dokument = clanek * 3  # Simulace delšího textu

print(f"📄 Zpracovávám dokument ({len(dlouhy_dokument)} znaků)...\n")
vysledek = sumarizuj_dlouhy_dokument(dlouhy_dokument)
print(f"\n📋 Finální souhrn:\n{vysledek}")

## 7. Automatické generování reportů

In [ ]:
import pandas as pd

def generuj_report(data_dict):
    """Generuje textový report z dat."""
    
    # Konverze dat na text
    text_parts = []
    for metrika, hodnota in data_dict.items():
        text_parts.append(f"The {metrika} is {hodnota}.")
    
    raw_text = " ".join(text_parts)
    
    # Generování interpretace
    prompt = f"Business report summary: {raw_text} In conclusion,"
    
    report = generator(prompt, max_length=150, do_sample=True, temperature=0.5)
    
    return report[0]['generated_text']

# Simulace business dat
mesicni_data = {
    "revenue": "$1.2 million, up 15% from last month",
    "customer satisfaction score": "4.5 out of 5",
    "new customers acquired": "250, exceeding target by 20%",
    "employee retention rate": "95%",
    "operational efficiency": "improved by 12%"
}

print("📊 MĚSÍČNÍ BUSINESS REPORT\n")
print("=" * 50)
print("\n📈 Klíčové metriky:")
for metrika, hodnota in mesicni_data.items():
    print(f"   • {metrika.title()}: {hodnota}")

print("\n📝 Automatický souhrn:")
report = generuj_report(mesicni_data)
print(report)

## 8. Parafrázování textu

In [ ]:
# PEGASUS pro parafrázování
paraphraser = pipeline("text2text-generation", 
                       model="tuner007/pegasus_paraphrase",
                       device=device)

originalni = "The company achieved significant growth in the last quarter due to increased market demand."

parafráze = paraphraser(originalni, num_return_sequences=3, num_beams=5, max_length=60)

print(f"📝 Originál: {originalni}\n")
print("🔄 Parafráze:")
for i, p in enumerate(parafráze, 1):
    print(f"   {i}. {p['generated_text']}")

---
## 🏁 Shrnutí

- ✅ BART a T5 pro sumarizaci
- ✅ GPT-2 pro generování textu
- ✅ Zpracování dlouhých dokumentů po částech
- ✅ Automatické generování reportů a newsletterů
- ✅ Parafrázování textu

**Další notebook:** Překlad a vícejazyčné modely